# Détection d'objet : version simplifiée de YOLO

<center> <img src="https://drive.google.com/uc?id=1V4aAS7K_Akj83apuMZ2vRjNvjgdgoOCh" width=500></center>
<caption><center> Pipeline de l'algorithme YOLO ([Redmon 2016]) </center></caption>

Dans ce TP, nous allons tenter d'aller un peu plus loin que le TP précédent en considérant le problème plus complexe de la détection d'objet, c'est-à-dire de la localisation et la classification conjointe de tous les objets dans l'image ; pour cela nous allons implémenter une version simplifiée de YOLO. Cette version est considérée simplifiée car ne reprenant pas l'intégralité des éléments décrite dans l'article de Redmon (par exemple, sur le choix de l'optimiseur). Une des simplifications principales est également que nous ne considérerons qu'un objet par cellule.

Pour rappel, l'idée de YOLO est de découper l'image en une grille de cellules et de réaliser une prédiction de plusieurs boîtes englobantes ainsi qu'une classification par cellule. La vidéo de la cellule suivante rappelle les concepts vus en cours sur YOLO et la détection d'objet en général.



In [ ]:
from IPython.display import IFrame
IFrame("https://video.polymny.studio/?v=012cd29c-db98-458f-80d3-6cc5c1da9be3/", width=640, height=360)

Récupération des données

In [ ]:
!git clone https://github.com/axelcarlier/wildlife.git


## Fonctions utiles

Définition des différentes variables utiles pour la suite

In [ ]:
IMAGE_SIZE = 64 # Dimension des images en entrée du réseau
CELL_PER_DIM = 8 # Nombre de cellules en largeur et en hauteur
BOX_PER_CELL = 1 # Nombre d'objets par cellule
NB_CLASSES = 4 # Nombre de classes du problème
PIX_PER_CELL = IMAGE_SIZE/CELL_PER_DIM

DATASET_SIZE = 376*4


Chargement des données et mise en forme pour le problème de détection. Les données qui posent problème (plus d'une boîte englobante par cellule) sont indiquées et affichées pendant le chargement.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

import PIL
from PIL import Image
import os, sys

import keras
from keras.utils import np_utils

def load_data_detection():
  # Chemin vers la base de données
  ds_path = "./wildlife/"
  # Chemins vers les données des 4 différentes classes
  paths = [ds_path+"buffalo/", ds_path+"elephant/", ds_path+"rhino/", ds_path+"zebra/"]
  # Indice d'ajout de données dans les variables x et y 
  i = 0
  # Préparation des structures de données pour x et y
  x = np.zeros((DATASET_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3))
  y = np.zeros((DATASET_SIZE, CELL_PER_DIM, CELL_PER_DIM, NB_CLASSES + 5*BOX_PER_CELL))

  # Sauvegarde des largeur/hauteur normalisées de bounding box
  widths = []
  heights = []

  # Parcours des chemins de chacune des classes
  for path in paths:

    # Parcours des fichiers (classés) du répertoire
    dirs = os.listdir(path)
    dirs.sort()

    for item in dirs:
      if os.path.isfile(path + item):
        # Extraction de l'extension du fichier 
        extension = item.split(".")[1]

        if extension=="jpg" or extension=="JPG":
          # Image : on va remplir la variable x
          # Lecture de l'image
          img = Image.open(path + item)
          #f, e = os.path.splitext(path+item)

          # Mise à l'échelle de l'image
          img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
          # Remplissage de la variable x
          x[i] = np.asarray(img, dtype=np.int32)

        elif extension=="txt":
          # Texte : coordonnées de boîtes englobantes pour remplir y
          labels = open(path + item, "r")
          # Récupération des lignes du fichier texte
          labels = labels.read().split('\n')
          # Si la dernière ligne est vide, la supprimer 
          if labels[-1]=="":
            del labels[-1]

          err_flag = 0
          boxes = []
          for label in labels:
            # Récupération des informations de la boîte englobante
            label = label.split()
            # Sauvegarde des largeur/hauteur de boîtes englobantes
            widths.append(float(label[3]))
            heights.append(float(label[4]))
            # Coordonnées du centre de la boîte englobante dans le repère image
            cx, cy = float(label[1]) * IMAGE_SIZE, float(label[2]) * IMAGE_SIZE
            # Détermination des indices de la cellule dans laquelle tombe le centre
            ind_x, ind_y = int(cx // PIX_PER_CELL), int(cy // PIX_PER_CELL)
            # YOLO : "The (x, y) coordinates represent the center of the box relative to the bounds of the grid cell."
            # On va donc calculer les coordonnées du centre relativement à la cellule dans laquelle il se situe
            cx_cell = (cx - ind_x * PIX_PER_CELL) / PIX_PER_CELL
            cy_cell = (cy - ind_y * PIX_PER_CELL) / PIX_PER_CELL
            # Indice de confiance de la boîte englobante
            presence = np.array([1], dtype="i")
            # "One-hot vector" représentant les probabilités de classe dans la cellule
            classes = np_utils.to_categorical(label[0], num_classes=4)
            # On range les probabilités de classe à la fin du vecteur ([ BOX 1 ; BOX 2 ; ... ; BOX N ; CLASSES])
            y[i, ind_x, ind_y, 5 * BOX_PER_CELL:] = classes

            boxes.append([cx, cy, label[3]*IMAGE_SIZE, label[4]*IMAGE_SIZE])
            # Détermination de l'indice de la boîte englobante de cellule dans laquelle ranger les informations
            ind_box = 0
            while y[i, ind_x, ind_y, 5*ind_box] == 1 and ind_box < BOX_PER_CELL - 1:
              # Si la boîte d'indice courant est déjà utilisée (présence = 1) 
              # et que l'on a pas atteint le nombre maximal de boîtes, on passe à la boîte suivante
              ind_box = ind_box + 1

            if y[i, ind_x, ind_y, 5*ind_box] == 1:
              print("ERREUR : LA CELLULE CONTIENT DEJA TOUTES LES BOITES DISPONIBLES")
              print(path + item)
              err_flag = 1
            else:
              y[i, ind_x, ind_y, 5*ind_box] = 1
              y[i, ind_x, ind_y, 5*ind_box + 1] = cx_cell
              y[i, ind_x, ind_y, 5*ind_box + 2] = cy_cell
              # Racine carrée de la largeur et hauteur de boîte
              y[i, ind_x, ind_y, 5*ind_box + 3] = math.sqrt(float(label[3]))
              y[i, ind_x, ind_y, 5*ind_box + 4] = math.sqrt(float(label[4]))

          i = i + 1
          if err_flag == 1:
            img_name = item.split(".")[0]
            img = Image.open(path + img_name + '.jpg')
            # Mise à l'échelle de l'image
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

            plt.imshow(img)
            for ind_cell in range(CELL_PER_DIM):                
              plt.plot([ind_cell*PIX_PER_CELL, ind_cell*PIX_PER_CELL], [0, IMAGE_SIZE-1], 'k-')
              plt.plot([0, IMAGE_SIZE-1], [ind_cell*PIX_PER_CELL, ind_cell*PIX_PER_CELL], 'k-')

            #for ind_y in range(CELL_PER_DIM):

            for ind_box_plot in range(len(boxes)):
              box = boxes[ind_box_plot]
              plt.plot(box[0], box[1], 'b.')
            plt.show()

        else:
          print("extension trouvée : ", extension)

  return x, y, widths, heights

x,y,w,h = load_data_detection()

Partage de la base de données entre données d'entraînement et de validation

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, random_state=42)

# Normalisation des images
x_train = x_train/255
x_val = x_val/255

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

Fonction d'affichage des données et des résultats de la détection.

In [ ]:
from scipy.special import softmax

def print_data_detection(x, y, id=None, image_size=IMAGE_SIZE, mode='gt'):
  if id==None:
    # Tirage aléatoire d'une image dans la base
    num_img = np.random.randint(x.shape[0]) 
    print(num_img)
  else:
    num_img = id

  img = x[num_img]
  lab = y[num_img]

  colors = ["blue", "yellow", "red", "orange"] # Différentes couleurs pour les différentes classes
  classes = ["Buffalo", "Elephant", "Rhino", "Zebra"]

  boxes = lab[:, :, 1:5]
  for ind_x in range(CELL_PER_DIM):
    for ind_y in range(CELL_PER_DIM):
      box = boxes[ind_x, ind_y]
      box[0] = box[0] * PIX_PER_CELL + ind_x * PIX_PER_CELL
      box[1] = box[1] * PIX_PER_CELL + ind_y * PIX_PER_CELL
      box[2] = box[2]**2 * IMAGE_SIZE
      box[3] = box[3]**2 * IMAGE_SIZE
      boxes[ind_x, ind_y] = box

  # Récupération de toutes les informations des boîtes englobantes
  all_presences = np.reshape(lab[:, :, 0], (CELL_PER_DIM*CELL_PER_DIM))
  all_boxes = np.reshape(lab[:, :, 1:5], (-1, 4))
  all_classes = np.reshape(lab[:, :, 5:9], (-1, 4))

  if mode=='pred':
    all_presences = 1 / (1 + np.exp(-all_presences))
    all_classes = softmax(all_classes, axis=1)

  indices_sorted = np.argsort(-all_presences)
  #print(all_presences[indices_sorted[0:5]])
  #print(all_classes[indices_sorted[0:5]])

  # Eliminer toutes les boîtes englobantes dont la probabilité de presence est < 0.5 
  seuil = 0.35
  all_boxes = all_boxes[np.where(all_presences > seuil)]
  all_classes = all_classes[np.where(all_presences > seuil)]
  all_presences = all_presences[np.where(all_presences > seuil)]


  # Affichage de l'image
  plt.imshow(img)
  for i in range(all_boxes.shape[0]):

    # Détermination de la classe
    class_id = np.argmax(all_classes[i])
    lab = all_boxes[i]
    #print("x: {}, y: {}, w: {}, h:{}".format(ax,ay,width, height))
    # Détermination des extrema de la boîte englobante
    p_x = [lab[0]-lab[2]/2, lab[0]+lab[2]/2]
    p_y = [lab[1]-lab[3]/2, lab[1]+lab[3]/2]
    # Affichage de la boîte englobante, dans la bonne couleur
    plt.plot([p_x[0], p_x[0]],p_y,color=colors[class_id])
    plt.plot([p_x[1], p_x[1]],p_y,color=colors[class_id])
    plt.plot(p_x,[p_y[0],p_y[0]],color=colors[class_id])
    plt.plot(p_x,[p_y[1],p_y[1]],color=colors[class_id], label=classes[class_id] + " " +  str(all_presences[i]))
    #plt.title("Vérité Terrain : Image {}".format(num_img, classes[class_id]))
  
  plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
  plt.show()  


print_data_detection(x_train, y_train, image_size=IMAGE_SIZE)



<center> <img src="https://drive.google.com/uc?id=1_wXc_gTIAr37STaxu3chq1EEjVSKv6a5" width=500></center>
<caption><center> Illustration de la couche de sortie de YOLO. </center></caption>

Le modèle que je vous propose ci-dessous n'est qu'une possibilité parmi beaucoup d'autres. L'article de Redmon mentionne une instabilité délicate pendant l'entraînement, ce qui m'a encouragé à choisir une fonction d'activation *elu* (*exponential linear unit*).

A vous de compléter la dernière couche pour avoir une sortie de la bonne dimension.

In [ ]:
import keras
from keras import layers
from keras import models
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout, Input
from keras.models import Model, Sequential
from keras import regularizers



def create_model_YOLO(input_shape=(64, 64, 3)):
  weight_decay = 0

  input_layer = Input(shape=input_shape)

  conv1 = Conv2D(32, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(input_layer)
  conv1 = Conv2D(32, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  conv1 = Conv2D(32, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  conv2 = Conv2D(64, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(64, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  conv2 = Conv2D(64, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  
  conv3 = Conv2D(128, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(128, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  conv3 = Conv2D(128, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

  dense4 = Flatten()(pool3)
  dense4 = Dense(512, activation='elu',kernel_regularizer=regularizers.l2(weight_decay))(dense4)
  dense5 = Dense(512, activation='elu',kernel_regularizer=regularizers.l2(weight_decay))(dense4)
  output = Dense(..., activation='linear',kernel_regularizer=regularizers.l2(weight_decay))(dense5) # A COMPLETER
  output = Reshape((...))(output) # A COMPLETER

  model = Model(input_layer, output)

  return model

In [ ]:
model = create_model_YOLO()
model.summary()

<center> <img src="https://drive.google.com/uc?id=1Fbt_Wh_BqZj8Pwt3-04325ItCkQp5G9X" style="width:500;height:300px;"></center>
<caption><center> Détail de la fonction de perte définie dans l'article YOLO v1 </center></caption>

Nous arrivons maintenant à la partie délicate de l'implémentation de YOLO : la définition de la fonction de coût à utiliser.

Comme nous l'avons vu dans le TP4, lorsque l'on écrit une fonction de coût personnalisée en Keras, il est nécessaire d'utiliser uniquement les fonctions présentes sur la page suivante : 
https://keras.rstudio.com/articles/backend.html

En effet cette fonction de coût qui sera appelée pendant l'entraînement traitera des tenseurs, et non des tableau *numpy*. On doit donc utiliser la librairie Tensorflow qui permet de manipuler les tenseurs.

Une partie essentielle de la fonction est déjà écrite : celle qui permet de séparer les données des cellules dites "vide" (la vérité terrain ne contient pas de boîte englobante) des "non vides".

Le détail de la fonction de coût est indiqué ci-dessus : dans l'article $\lambda_{\text{coord}} = 5$ et $\lambda_{\text{noobj}} = 0.5$. Les $x_i$, $y_i$, $w_i$, $h_i$ correspondent aux coordonnées d'une boîte englobante, $C_i$ correspond à la probabilité de présence d'un objet dans la cellule (fonction sigmoïde appliquée aux éléments de sortie correspondant), et les $p_i(c)$ sont les probabilités de classe (fonction softmax appliquée aux éléments de sortie correspondant).

A vous de compléter l'expression des sous-fonctions de la fonction de coût (les fonctions *K.sum*, *K.square*, *K.sigmoid* et *K.softmax* devraient vous suffire !). 

**NB1 : Notez qu'ici, on choisit pour simplifier l'implémentation d'appliquer les fonction sigmoide et softmax directement dans la fonction de coût aux sorties correspondantes, plutôt que dans la couche de sortie du réseau.**

**NB2 : cette implémentation de la fonction de coût est très simplifiée et prend en compte le fait qu'il n'y a qu'une seule boîte englobante par cellule.**

In [ ]:
from keras import backend as K

# Définition de la fonction de perte YOLO
def YOLOss(lambda_coord, lambda_noobj, batch_size):

    # Partie "verte" : sous-partie concernant l'indice de confiance et les 
    # probabilités de classe dans le cas où une boîte est présente dans la cellule
    def box_loss(y_true, y_pred):
      return ... # A COMPLETER

    # Partie "bleue" : sous-partie concernant les coordonnées de boîte englobante 
    # dans le cas où une boîte est présente dans la cellule
    def coord_loss(y_true, y_pred):
      return ... # A COMPLETER


    # Partie "rouge" : sous-partie concernant l'indice de confiance  
    # dans le cas où aucune boîte n'est présente dans la cellule
    def nobox_loss(y_true, y_pred):
      return ... # A COMPLETER


    def YOLO_loss(y_true, y_pred):

      # On commence par reshape les tenseurs de bs x S x S x (5B+C) à (bsxSxS) x (5B+C)
      y_true = K.reshape(y_true, shape=(-1, 9))
      y_pred = K.reshape(y_pred, shape=(-1, 9))

      # On cherche (dans les labels y_true) les indices des cellules pour lesquelles au moins la première boîte englobante est présente
      not_empty = K.greater_equal(y_true[:, 0], 1)      
      indices = K.arange(0, K.shape(y_true)[0])
      indices_notempty_cells = indices[not_empty]

      empty = K.less_equal(y_true[:, 0], 0)
      indices_empty_cells = indices[empty]

      # On sépare les cellules de y_true et y_pred avec ou sans boîte englobante
      y_true_notempty = K.gather(y_true, indices_notempty_cells)
      y_pred_notempty = K.gather(y_pred, indices_notempty_cells)

      y_true_empty = K.gather(y_true, indices_empty_cells)
      y_pred_empty = K.gather(y_pred, indices_empty_cells)

      return (box_loss(y_true_notempty, y_pred_notempty) + lambda_coord*coord_loss(y_true_notempty, y_pred_notempty) + lambda_noobj*nobox_loss(y_true_empty, y_pred_empty))/batch_size

   
    # Return a function
    return YOLO_loss

In [ ]:
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

batch_size=16
model = create_model_YOLO()
opt = Adam(learning_rate=1e-4)  

# Comme l'entraînement est instable, on déclenche une sauvegarde du modèle à chaque fois que
# la perte de validation atteint un nouveau minimum
model_saver = ModelCheckpoint('tmp/best_weights', monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

loss=[YOLOss(5, 0.5, batch_size)]

model.compile(loss=loss,
              optimizer=opt)

history = model.fit(x_train, y_train,
              epochs=100,
              batch_size=batch_size,           
              validation_data=(x_val, y_val),
              callbacks = [model_saver])


Test de la version à la fin de l'entrainement

In [ ]:
y_pred = model.predict(x_train)
print_data_detection(x_train, y_pred, image_size=IMAGE_SIZE, mode='pred')

In [ ]:
y_pred = model.predict(x_val)
print_data_detection(x_val, y_pred, image_size=IMAGE_SIZE, mode='pred')

Test de la meilleure version sauvegardée

In [ ]:
model.load_weights('tmp/best_weights')

In [ ]:
y_pred = model.predict(x_train)
print_data_detection(x_train, y_pred, image_size=IMAGE_SIZE, mode='pred')

In [ ]:
y_pred = model.predict(x_val)
print_data_detection(x_val, y_pred, image_size=IMAGE_SIZE, mode='pred')

## Chargement de poids d'un réseau déjà entraîné

L'entraînement de YOLO étant très instable, il est possible qu'à l'issue du TP vous n'obteniez pas des résultats très probants. Pour finir ce TP, je vous propose de charger les poids d'un modèle que j'ai entraîné pendant un long moment et de visualiser les résultats.

In [ ]:
# Téléchargement des poids
!wget https://drive.google.com/uc?id=1PtOtf4Du69Sqzj3oYS2nD1mAoz8n1KgZ -O best_weights.index
!wget https://drive.google.com/uc?id=1w9VHJxjOEUIhcJZeUBkIl7ZnTJLYz9kv -O best_weights.data-00000-of-00001

In [ ]:
model.load_weights('best_weights')

In [ ]:
y_pred = model.predict(x_train)
print_data_detection(x_train, y_pred, image_size=IMAGE_SIZE, mode='pred')

In [ ]:
y_pred = model.predict(x_val)
print_data_detection(x_val, y_pred, image_size=IMAGE_SIZE, mode='pred')

Les résultats ne sont certes pas parfaits mais on commence à voir apparaître quelques résultats satisfaisants. Le modèle a sur-appris, je n'ai pas intégré l'augmentation de données dans cet entraînement (cela aurait pu aider) mais on peut voir sur les quelques exemples ci-dessous que certaines des images sont plutôt bien prédites.

In [ ]:
y_pred = model.predict(x_val)

print_data_detection(x_val, y_pred, id=81, image_size=IMAGE_SIZE, mode='pred')
print_data_detection(x_val, y_pred, id=28, image_size=IMAGE_SIZE, mode='pred')
print_data_detection(x_val, y_pred, id=37, image_size=IMAGE_SIZE, mode='pred')
print_data_detection(x_val, y_pred, id=220, image_size=IMAGE_SIZE, mode='pred')
print_data_detection(x_val, y_pred, id=214, image_size=IMAGE_SIZE, mode='pred')
print_data_detection(x_val, y_pred, id=193, image_size=IMAGE_SIZE, mode='pred')
print_data_detection(x_val, y_pred, id=39, image_size=IMAGE_SIZE, mode='pred')
print_data_detection(x_val, y_pred, id=108, image_size=IMAGE_SIZE, mode='pred')
